In [1]:
import os
import time
import requests
try:
    import pyarrow
except:
    %pip install pyarrow
    import pyarrow
try:
    import bs4
    from bs4 import BeautifulSoup
except:
    %pip install beautifulsoup4
    import bs4
    from bs4 import BeautifulSoup
try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd

from urllib.parse import urlparse

import re

In [2]:
headers = {
        "User-Agent": "Mozilla/5.0 AppleWebKit/605.1.15 Version/17.4.1 Safari/605.1.15",
#        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8",
        "Accept-Language": "fr-FR,fr;q=0.8",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

In [3]:
url = "https://candidat.francetravail.fr/offres/recherche?motsCles=data+engineer&offresPartenaires=true&rayon=10&tri=0"
r = requests.get(url, headers= headers)

In [4]:
r.headers

{'Cache-Control': 'no-cache', 'Date': 'Wed, 12 Mar 2025 14:12:22 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'X-OneAgent-JS-Injection': 'true', 'X-Content-Type-Options': "nosniff'", 'Set-Cookie': 'dtCookie=v_4_srv_9_sn_0AAE453D4B62125021FD8486094B172C_perc_100000_ol_0_mul_1_app-3A6e350f5c3735afd9_1_rcs-3Acss_0; Path=/; Domain=.francetravail.fr; Secure; , JSESSIONID_RECH_OFFRE=FyWKswQij28gldtumwL9HOXzBF1Fhv9Le_Coh3FKtVnYYcBQn303!-1413685851; path=/; HttpOnly; Secure; , userBadge=0; domain=.francetravail.fr; expires=Wed, 12-Mar-2025 16:12:22 GMT; path=/; secure, BIGipServerPOOL_PWNOT-00PT28_HTTPS_PN055-VIPA_3_PN055=3594324490.56131.0000; path=/; Httponly; Secure, TS016fc3b0=0150c672c3917210aeb132e9a4b58604a029fd3823184692c7d7133b90519c242457caa9219ab68e26046a2bab241136644b00b314; Path=/; Domain=.candidat.francetravail.fr; ', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked'}

In [5]:

with open("FT.html", "w") as f:
    f.write(BeautifulSoup(r.text).prettify())

In [6]:
soup = BeautifulSoup(r.text)

In [7]:
urls = [link.get("href") for link in soup.select('ul[data-container-type="zone"] > li > a')]

In [8]:
urls

['/offres/recherche/detail/189KZNQ',
 '/offres/recherche.rechercheoffre.elementoffredelaliste:demandeauthentajouteralaselectiondepuisliste/189KZNQ?motsCles=data+engineer&offresPartenaires=true&rayon=10&tri=0',
 '/offres/recherche/detail/189GQNP',
 '/offres/recherche.rechercheoffre.elementoffredelaliste:demandeauthentajouteralaselectiondepuisliste/189GQNP?motsCles=data+engineer&offresPartenaires=true&rayon=10&tri=0',
 '/offres/recherche/detail/189GJVT',
 '/offres/recherche.rechercheoffre.elementoffredelaliste:demandeauthentajouteralaselectiondepuisliste/189GJVT?motsCles=data+engineer&offresPartenaires=true&rayon=10&tri=0',
 '/offres/recherche/detail/188ZWDF',
 '/offres/recherche.rechercheoffre.elementoffredelaliste:demandeauthentajouteralaselectiondepuisliste/188ZWDF?motsCles=data+engineer&offresPartenaires=true&rayon=10&tri=0',
 '/offres/recherche/detail/188ZDGF',
 '/offres/recherche.rechercheoffre.elementoffredelaliste:demandeauthentajouteralaselectiondepuisliste/188ZDGF?motsCles=data

In [ ]:
domaine = "https://candidat.francetravail.fr/offres/recherche/detail/189KZNQ"

In [10]:
for i in range(0,len(urls), 2):
    print(urls[i])
    r = requests.get(domaine + str(urls[i]), headers=headers)
    break

/offres/recherche/detail/189KZNQ


In [ ]:
competences = []
savoirs = []
soup = BeautifulSoup(r.text)

def parse_soup(soup: bs4.BeautifulSoup):
    annonce = dict()
    annonce["intitule_poste"] = soup.find("span", {"itemprop": "title"}).text
    annonce["description"] = soup.find("div", {"itemprop": "description"}).text
    annonce["experience"] = soup.find("span", {"itemprop": "experienceRequirements"}).text
    skill = soup.select("ul.skill-list.list-unstyled > li")


    for item in skill:
        exp = item.find_all("span", {"itemprop": "experienceRequirements"})
        educ = item.find_all("span", {"itemprop": "educationRequirements"})
        comp = item.find("span", {"class": "skill skill-competence"})
        sav = item.find("span", {"class": "skill skill-savoir"})
        if len(exp) != 0:
            experience = exp[0].text
            exp_requis = True if item.find("span", {"class": "skill-required"}) else False
            annonce["experience"] = {"exp": experience, "requis": exp_requis}
        if education:
            education = educ[0].text
            educ_requis = True if item.find("span", {"class": "skill-required"}) else False
            annonce["experience"] = {"exp": education, "requis": educ_requis}
        
        if comp:
            competence = comp.find("span", {"class": "skill-name"}).text
            requis = True if comp.find("span", {"class": "skill-required"}) else False
            competences.append((competence, requis))
        if sav:
            savoir = sav.find("span", {"class": "skill-name"}).text
            requis = True if sav.find("span", {"class": "skill-required"}) else False
            savoirs.append((savoir, requis))
    annonce["compétences"] = competences
    annonce["savoirs"] = savoirs
    infos = soup.find("div", {"class": "description-aside col-sm-4 col-md-5"}).select("dd")
    type_contrat = infos[0].text
    hebdo = infos[1].text
    divers = [item.text for item in infos[2:]]
    annonce["type_contrat"] = type_contrat
    annonce["heures_hebdo"] = hebdo
    annonce["divers"] = divers
    return annonce
annonce = parse_soup(soup)

In [67]:
annonce

{'intitule_poste': 'Data engineer (H/F)',
 'description': "IONE Talents & Technology, société de Conseil en Système d'Information avec une forte expertise métier créée par un expert dans le domaine de la data, IONE Talents & Technology c'est surtout un ensemble de consultant.e.s capables d'intervenir à tous les niveaux d'un Système d'Information.\n\nSi vous souhaitez rejoindre une entreprise à taille humaine où chaque employé peut avoir un réel rôle à jouer dans son développement, vous êtes au bon endroit.\n\nDans le cadre de notre croissance sur la région d'ile de France, nous recrutons actuellement un(e) Data engineer IT en CDI pour pour intervenir chez nos clients dans la mise en place de nouveaux projets.\n\n\nProfil recherché : \n- Expérience significative dans un environnement Cloud (AWS, Azure, GCP) \n- Pratique confirmée des méthodes Agiles \n Compétences techniques \n- Maîtrise des langages de programmation : Python 3, Scala, Spark \n- Expertise en technologies Big Data : Hado

In [49]:
import psycopg2

try:
    conn = psycopg2.connect(
        dbname="webmining",
        user="postgres",
        password="postgres",
        host="localhost",  # If connecting from outside Docker
        port="5432"
    )
    print("✅ Connection successful!")
    conn.close()
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connection successful!


In [59]:
import psycopg2

conn = psycopg2.connect(
    dbname="webmining",
    user="postgres",
    password="postgres",
    host="localhost",  # Change if using Docker
    port="5432"
)

cur = conn.cursor()

# Drop all tables in the public schema
drop_tables_sql = """
DO $$ 
DECLARE
    r RECORD;
BEGIN
    FOR r IN (SELECT tablename FROM pg_tables WHERE schemaname = 'public') LOOP
        EXECUTE 'DROP TABLE IF EXISTS public.' || r.tablename || ' CASCADE';
    END LOOP;
END $$;
"""

cur.execute(drop_tables_sql)
conn.commit()

print("✅ All tables dropped successfully!")

cur.close()
conn.close()


✅ All tables dropped successfully!


In [60]:
import psycopg2

conn = psycopg2.connect(
    dbname="webmining",
    user="postgres",
    password="postgres",
    host="localhost",  # Change if using Docker
    port="5432"
)

cur = conn.cursor()


# Table creation queries
with open("db_webmining.sql", "r") as f:
    create_tables_sql = f.read()

cur.execute(create_tables_sql)
conn.commit()

print("✅ Tables created successfully!")

cur.close()
conn.close()


✅ Tables created successfully!


In [61]:
import psycopg2

# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    dbname="webmining",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cur = conn.cursor()



# Insertion des données dans la table postes
# Insert competences (if not already inserted)
for competence, requis in competences:
    cur.execute("""
        INSERT INTO competences (nom, requis)
        VALUES (%s, %s)
        ON CONFLICT (nom) DO NOTHING;
    """, (competence, requis))

# Insertion des savoirs
for savoir, requis in savoirs:
    cur.execute("""
        INSERT INTO savoirs (nom, requis)
        VALUES (%s, %s, %s);
    """, (savoir, requis))

# Insert postes (job positions)
cur.execute("""
    INSERT INTO postes (intitule_poste, description, experience, type_contrat, hebdo, divers)
    VALUES (%s, %s, %s, %s, %s, %s)
    RETURNING id;
""", (intitule_poste, description, experience, type_contrat, hebdo, divers))

poste_id = cur.fetchone()[0]

for competence, requis in competences:
    # Insert relationship into the junction table
    cur.execute("""
        INSERT INTO poste_competences (poste_id, competence_id)
        SELECT %s, id FROM competences WHERE nom = %s;
    """, (poste_id, competence))

for savoir, requis in competences:
    cur.execute("""
        INSERT INTO poste_savoirs (poste_id, savoirs_id)
        SELECT %s, id FROM savoirs WHERE nom = %s;
    """, (poste_id, savoir))



# Validation des modifications et fermeture de la connexion
conn.commit()
cur.close()
conn.close()


In [63]:
import psycopg2

# Connexion à la base de données PostgreSQL
conn = psycopg2.connect(
    dbname="webmining",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Define the poste_id you want to query for (e.g., 1)
poste_id = 1

# SQL Query to retrieve competences for a specific poste
query = '''
SELECT p.intitule_poste, c.nom AS competence, c.requis
FROM postes p
JOIN poste_competences pc ON p.id = pc.poste_id
JOIN competences c ON pc.competence_id = c.id
WHERE p.id = %s;
'''

# Execute the query with the specific poste_id
cur.execute(query, (poste_id,))

# Fetch the results
competences = cur.fetchall()

# Print the results
for row in competences:
    print(f"Poste: {row[0]}, Competence: {row[1]}, Requis: {row[2]}")

# Close the cursor and connection
cur.close()
conn.close()


Poste: Data engineer (H/F), Competence: Actualiser régulièrement ses connaissances, Requis: True
Poste: Data engineer (H/F), Competence: Adapter les méthodes de travail à la réglementation locale, Requis: True
Poste: Data engineer (H/F), Competence: Algorithmique, Requis: True
Poste: Data engineer (H/F), Competence: Analyser / traiter l'information à des fins d'anticipation, Requis: True
Poste: Data engineer (H/F), Competence: Analyser, exploiter, structurer des données, Requis: True
Poste: Data engineer (H/F), Competence: Anglais technique, Requis: True
Poste: Data engineer (H/F), Competence: Business Intelligence (BI) / Informatique décisionnelle, Requis: True
Poste: Data engineer (H/F), Competence: Collaborer dans un groupe pour réaliser un projet, Requis: True
Poste: Data engineer (H/F), Competence: Communiquer à l'écrit de façon appropriée, Requis: True
Poste: Data engineer (H/F), Competence: Concevoir et gérer un projet, Requis: True
Poste: Data engineer (H/F), Competence: Convai